In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys; sys.path.insert(0, '..')

In [3]:
from pathlib import Path

import fiona
import geopandas as gpd
import numpy as np
import osmnx as ox
import pandas as pd
from shapely.geometry import LineString, Point
from tqdm import tqdm
import altair as alt

In [4]:
from main import prepare_data_for_place, OUTPUT_COLUMNS
from src.route import (
    get_route_gdf,
    compute_routes_from_census_blocks_to_school,
    compute_routes_from_census_blocks_to_all_schools
)

## Prepare out dir

In [5]:
import os
import shutil

OUT_PATH = Path("../data/out/notebook/")

# Delete the directory if it exists
if OUT_PATH.exists():
    shutil.rmtree(OUT_PATH)

# Recreate the directory
OUT_PATH.mkdir(parents=True, exist_ok=True)

## Load bike network

In [6]:
place = "Somerville, MA, USA"
nodes, edges = prepare_data_for_place(place)

> Getting bike network for Somerville, MA, USA
> Processing network for Somerville, MA, USA
> MODEL 1: Preparing speed data for Somerville, MA, USA
> MODEL 2: Preparing separation level data for Somerville, MA, USA
> MODEL 3: Preparing street category data for Somerville, MA, USA
> MODEL 4: Preparing lanes data for Somerville, MA, USA
> MODEL: Preparing composite score for Somerville, MA, USA


In [7]:
edges = edges[OUTPUT_COLUMNS]

In [8]:
G = ox.graph_from_gdfs(nodes, edges)

In [9]:
edges.sample(3)

,,,name,maxspeed_0,maxspeed_int,maxspeed_int_score,separation_level,separation_level_score,street_0,street_classification,street_classification_score,composite_score,length,width_float,width_half,geometry
u,v,key,,,,,,,,,,,,,,
1183,814,0,McGrath Highway,40 mph,40.0,3.0,track,1.0,trunk,motorway,4,2.0,168.437330,45.7,22.85,"LINESTRING (-71.08931 42.38848, -71.0892 42.38..."
25,1549,0,Grove Street,20 mph,20.0,0.0,shared_lane,3.5,residential,residential,2,2.5,33.640190,12.2,6.1,"LINESTRING (-71.12066 42.39543, -71.12085 42.3..."
1287,1456,0,Garfield Avenue,NaN,NaN,NaN,none,4.0,unclassified,residential,2,3.5,54.420375,10.0,5.0,"LINESTRING (-71.08455 42.39103, -71.08458 42.3..."


## Load Schools

In [10]:
school_gdb_path = "../data/raw/SafeRoutesGISLayers.gdb.zip"

In [11]:
layers = fiona.listlayers(school_gdb_path)
layers

['SafetyZoneStreets', 'PublicSchools', 'Sidewalks', 'Signalized_Intersections']

In [12]:
# read school data
schools_gdf = gpd.read_file(school_gdb_path, layer='PublicSchools')

# save schools polygons
schools_gdf.to_file((OUT_PATH / "schools_poly.gpkg"), driver="GPKG")

# make geom col into centroids
schools_gdf['geometry'] = schools_gdf.centroid

# save schools polygons
schools_gdf.to_file((OUT_PATH / "schools_centroid.gpkg"), driver="GPKG")

In [13]:
schools_gdf.head(3)

,Name,GlobalID,Shape_Length,Shape_Area,geometry
0,West Somerville Neighborhood School,{423648E4-357B-4C51-8323-18DE5B5EF869},857.126130,20546.222891,POINT (757029.484 2973287.291)
1,Brown School,{32ED129B-38AE-4E8F-A71B-A18126973D75},511.378543,10156.639765,POINT (760400.444 2970061.762)
2,Healey School,{374CFA80-E38D-4411-AB46-7868E8DA8468},900.132189,38897.272280,POINT (765459.28 2970148.61)


## Load census blocks

In [14]:
# read census blocks
census_blocks = gpd.read_file("../data/raw/Census_2020_Blocks.zip")

# filter by TOWN attribute
somerville_census_blocks = census_blocks[census_blocks['TOWN'] == "SOMERVILLE"].copy()

# reset index
somerville_census_blocks = somerville_census_blocks.reset_index(drop=True)

# get a sample
half_n_census_blocks = len(somerville_census_blocks) // 4
somerville_census_sample = somerville_census_blocks.sample(half_n_census_blocks)

# save polygon version
somerville_census_blocks.to_file((OUT_PATH / "somer_blocks_poly.gpkg"), driver="GPKG")
somerville_census_sample.to_file((OUT_PATH / "somer_sample_poly.gpkg"), driver="GPKG")

# convert geometry to centroid
somerville_census_blocks['geometry'] = somerville_census_blocks.centroid
somerville_census_sample['geometry'] = somerville_census_sample.centroid

# save centroid version
somerville_census_blocks.to_file((OUT_PATH / "somer_blocks_centroid.gpkg"), driver="GPKG")
somerville_census_sample.to_file((OUT_PATH / "somer_sample_centroid.gpkg"), driver="GPKG")

In [15]:
somerville_census_blocks.head(3)

,OBJECTID,STATEFP20,COUNTYFP20,TRACTCE20,BLOCKCE20,GEOID20,NAME20,MTFCC20,ALAND20,AWATER20,...,AREA_SQFT,AREA_ACRES,TOWN,TOWN_ID,BLKGRP20,TRACT20,COUSUBFP,SHAPEAREA,SHAPELEN,geometry
0,51243,25,017,351002,2002,250173510022002,Block 2002,G5040,18175,0,...,195621.53,4.49,SOMERVILLE,274,250173510022,25017351002,62535,18173.907063,626.570381,POINT (231829.87 904798.558)
1,51295,25,017,350400,2007,250173504002007,Block 2007,G5040,17571,0,...,189123.23,4.34,SOMERVILLE,274,250173504002,25017350400,62535,17570.193552,675.342228,POINT (231953.871 905367.479)
2,51456,25,017,350108,2003,250173501082003,Block 2003,G5040,19544,0,...,210356.28,4.83,SOMERVILLE,274,250173501082,25017350108,62535,19542.816299,707.710510,POINT (233007.237 905138.728)


## Make sure everything has same crs

- EPSG:26986 =  NAD83 / Massachusetts Mainland Meters
- EPSG:4326 = WGS 84 / web

In [16]:
def crs_first_line(gdf):
    return str(gdf.crs).splitlines()[0]

In [17]:
print("somerville_census_blocks:", crs_first_line(somerville_census_blocks))
print("somerville_census_sample:", crs_first_line(somerville_census_sample))
print("schools_gdf             :", crs_first_line(schools_gdf))
print("edges                   :", crs_first_line(edges))
print("nodes                   :", crs_first_line(nodes))

somerville_census_blocks: EPSG:26986
somerville_census_sample: EPSG:26986
schools_gdf             : EPSG:6492
edges                   : EPSG:4326
nodes                   : EPSG:4326


In [18]:
# use this one
use_crs = edges.crs

# make them match
somerville_census_blocks = somerville_census_blocks.to_crs(use_crs)
somerville_census_sample = somerville_census_sample.to_crs(use_crs)
schools_gdf = schools_gdf.to_crs(use_crs)
nodes = nodes.to_crs(use_crs)
edges = edges.to_crs(use_crs)

In [19]:
print("somerville_census_blocks:", crs_first_line(somerville_census_blocks))
print("somerville_census_sample:", crs_first_line(somerville_census_sample))
print("schools_gdf             :", crs_first_line(schools_gdf))
print("edges                   :", crs_first_line(edges))
print("nodes                   :", crs_first_line(nodes))

somerville_census_blocks: EPSG:4326
somerville_census_sample: EPSG:4326
schools_gdf             : EPSG:4326
edges                   : EPSG:4326
nodes                   : EPSG:4326


## Routing

Try routing from a single block centroid to a single school

In [20]:
G = ox.graph_from_gdfs(nodes, edges)

In [21]:
# pick a school
dest_point = schools_gdf.loc[0, 'geometry']

In [22]:
# pick a census centroid
orig_point = somerville_census_sample.iloc[0]['geometry']

In [23]:
# route based on composite score
route_gdf = get_route_gdf(G, orig_point, dest_point, weight="composite_score")
route_gdf

,geometry,weighted_mean_score,min_score,max_score,sum_length
0,"LINESTRING (-71.09651 42.39664, -71.09676 42.3...",2.161621,0.75,3.5,3279.140082


In [24]:
# route based on length
route_gdf = get_route_gdf(G, orig_point, dest_point, weight="length")
route_gdf

,geometry,weighted_mean_score,min_score,max_score,sum_length
0,"LINESTRING (-71.09651 42.39664, -71.09676 42.3...",1.966063,0.75,3.75,3110.845439


## Route loop

Try routing from all block centroids to a single school

In [25]:
# loop over all census blocks, computing routes to one school
combined_gdf, errors = compute_routes_from_census_blocks_to_school(
    G, somerville_census_blocks,
    schools_gdf.loc[0],
    weight="composite_score"
)

100%|████████████████████████████████████████████████████████████████████████████████████| 702/702 [00:06<00:00, 114.93it/s]


In [26]:
combined_gdf.head(3)

,geometry,weighted_mean_score,min_score,max_score,sum_length,from_block_geoid,from_blkgrp20,from_tract20,to_school_name,to_school_id
0,"LINESTRING (-71.114 42.39291, -71.11489 42.393...",2.285886,0.00,3.50,2500.611909,250173510022002,250173510022,25017351002,West Somerville Neighborhood School,{423648E4-357B-4C51-8323-18DE5B5EF869}
1,"LINESTRING (-71.11089 42.39896, -71.11136 42.3...",2.044498,0.75,2.80,1564.153473,250173504002007,250173504002,25017350400,West Somerville Neighborhood School,{423648E4-357B-4C51-8323-18DE5B5EF869}
2,"LINESTRING (-71.09862 42.39624, -71.0978 42.39...",2.201702,0.75,3.75,3058.507568,250173501082003,250173501082,25017350108,West Somerville Neighborhood School,{423648E4-357B-4C51-8323-18DE5B5EF869}


In [27]:
# save only the combined file
combined_gdf.to_file(OUT_PATH / "routes_to_school_0_by_risk.gpkg", driver="GPKG")

## Loop all

Route all block centroids to all schools

### Loop all schools - by risk

In [28]:
all_routes_gdf_by_risk, errors = compute_routes_from_census_blocks_to_all_schools(
    G,
    somerville_census_blocks=somerville_census_blocks,
    schools_gdf=schools_gdf,
    weight="composite_score"
)

----- West Somerville Neighborhood School -----


100%|████████████████████████████████████████████████████████████████████████████████████| 702/702 [00:06<00:00, 113.19it/s]


----- Brown School -----


100%|████████████████████████████████████████████████████████████████████████████████████| 702/702 [00:06<00:00, 114.80it/s]


----- Healey School -----


...

100%|████████████████████████████████████████████████████████████████████████████████████| 702/702 [00:05<00:00, 121.31it/s]


----- Somerville High School -----


100%|████████████████████████████████████████████████████████████████████████████████████| 702/702 [00:05<00:00, 119.07it/s]


----- Winter Hill at Edgerly -----


100%|████████████████████████████████████████████████████████████████████████████████████| 702/702 [00:06<00:00, 110.53it/s]


In [29]:
# save only the combined file
all_routes_gdf_by_risk.to_file(OUT_PATH / "routes_by_risk.gpkg", driver="GPKG")

In [30]:
all_routes_gdf_by_length, errors = compute_routes_from_census_blocks_to_all_schools(
    G,
    somerville_census_blocks=somerville_census_blocks,
    schools_gdf=schools_gdf,
    weight="length"
)

----- West Somerville Neighborhood School -----


100%|█████████████████████████████████████████████████████████████████████████████████████| 702/702 [00:07<00:00, 93.72it/s]


----- Brown School -----


100%|████████████████████████████████████████████████████████████████████████████████████| 702/702 [00:06<00:00, 103.66it/s]


----- Healey School -----


...

100%|████████████████████████████████████████████████████████████████████████████████████| 702/702 [00:06<00:00, 115.27it/s]


----- Somerville High School -----


100%|████████████████████████████████████████████████████████████████████████████████████| 702/702 [00:05<00:00, 117.08it/s]


----- Winter Hill at Edgerly -----


100%|████████████████████████████████████████████████████████████████████████████████████| 702/702 [00:06<00:00, 104.59it/s]


In [31]:
# save only the combined file
all_routes_gdf_by_length.to_file(OUT_PATH / "routes_by_length.gpkg", driver="GPKG")

## Summarize

Combine into one df, drop geometry

In [32]:
all_routes_gdf_by_length['method'] = "length"
all_routes_gdf_by_risk['method'] = "risk"

all_routes_gdf = pd.concat([
    all_routes_gdf_by_length.drop(columns=['geometry']),
    all_routes_gdf_by_risk.drop(columns=['geometry'])
])

In [33]:
school_means = all_routes_gdf.groupby(['school_name', 'method']).mean(numeric_only=True).reset_index()
school_means

,school_name,method,weighted_mean_score,min_score,max_score,sum_length
0,Argenziano School,length,2.329820,0.821633,3.350466,2712.129071
1,Argenziano School,risk,1.436578,0.142407,3.210781,3863.948236
2,Brown School,length,2.130473,0.923317,3.237536,2295.203194
3,Brown School,risk,1.459696,0.456232,3.107521,3050.975476
4,Capuano Early Childhood Center,length,2.366698,1.038020,3.320373,2725.532959
5,Capuano Early Childhood Center,risk,1.392559,0.263343,3.189455,3543.083254
6,East Somerville Community School,length,2.603324,1.513486,3.545696,2463.837753
7,East Somerville Community School,risk,1.776562,0.585653,3.580057,3049.192624
8,Healey School,length,2.442303,1.027865,3.682987,2290.121721
9,Healey School,risk,1.896713,0.496991,3.413503,3492.010704


#### Sort by difference

- median_composite_score: $length - risk$

In [34]:
# reset
school_means = all_routes_gdf.groupby(['school_name', 'method']).mean(numeric_only=True).reset_index()

# pivot to get length and risk methods as separate columns
pivoted = school_means.pivot(
    index='school_name', 
    columns='method', 
    values='weighted_mean_score'
).reset_index()

# calculate the difference (length - risk)
pivoted['mean_score_diff'] = pivoted['length'] - pivoted['risk']

# merge back with the original data
school_means = school_means.merge(
    pivoted[['school_name', 'mean_score_diff']], 
    on='school_name'
).sort_values('mean_score_diff', ascending=False)

school_means.head()

,school_name,method,weighted_mean_score,min_score,max_score,sum_length,mean_score_diff
13,Somerville High School,risk,1.148861,0.000000,3.067371,2566.059589,0.987881
12,Somerville High School,length,2.136742,0.000000,3.256913,1963.662643,0.987881
4,Capuano Early Childhood Center,length,2.366698,1.038020,3.320373,2725.532959,0.974139
5,Capuano Early Childhood Center,risk,1.392559,0.263343,3.189455,3543.083254,0.974139
0,Argenziano School,length,2.329820,0.821633,3.350466,2712.129071,0.893242


In [35]:
school_means.to_csv(OUT_PATH / "school_means.csv")

In [36]:
# Create the bar chart with sorting by mean_score_diff
chart = alt.Chart(school_means).mark_bar().encode(
    x=alt.X('school_name:N', 
            title='School', 
            axis=alt.Axis(labelAngle=-45),
            sort=alt.EncodingSortField(field='mean_score_diff', op='max', order='descending')),
    y=alt.Y('weighted_mean_score:Q', title='Weighted Mean Score'),
    color=alt.Color('method:N', title='Method', scale=alt.Scale(scheme='set2')),
    xOffset='method:N',  # This creates grouped bars
    tooltip=[
        alt.Tooltip('school_name:N', title='School'),
        alt.Tooltip('method:N', title='Method'),
        alt.Tooltip('weighted_mean_score:Q', title='Weighted Mean Score', format='.3f'),
        alt.Tooltip('sum_length:Q', title='Total Length', format='.1f')
    ]
).properties(
    width=600,
    height=400,
    title='Mean Composite Score by School and Method (Risk vs. Length)'
)

# Get one row per school for the diff labels (take the first method's row since diff is the same)
school_diff = school_means.groupby('school_name').agg({
    'mean_score_diff': 'first',
    'weighted_mean_score': 'max'  # Use max to position text above tallest bar
}).reset_index()

# Add text labels centered above both bars showing mean_score_diff
text = alt.Chart(school_diff).mark_text(
    align='center',
    baseline='bottom',
    dy=-5,
    fontSize=10,
    fontWeight='bold'
).encode(
    x=alt.X('school_name:N', sort=alt.EncodingSortField(field='mean_score_diff', op='max', order='descending')),
    y=alt.Y('weighted_mean_score:Q'),
    text=alt.Text('mean_score_diff:Q', format='.3f')
)

# Combine the chart and text
final_chart = chart + text
final_chart

alt.LayerChart(...)

In [37]:
# save HTML to upload dir
final_chart.save(OUT_PATH / "chart1.html")

### Compare blocks

In [38]:
block_means = all_routes_gdf.groupby(['from_block_geoid', 'method']).mean(numeric_only=True).reset_index()
block_means.head()

,from_block_geoid,method,weighted_mean_score,min_score,max_score,sum_length
0,250173501051001,length,2.684740,0.000000,3.616667,2632.464825
1,250173501051001,risk,2.358446,0.000000,3.888889,3345.271055
2,250173501051002,length,2.613282,1.150000,3.722222,2641.969993
3,250173501051002,risk,2.288285,0.438889,3.833333,3435.936738
4,250173501051003,length,2.582950,0.816667,3.722222,2748.514220


#### Get difference

In [39]:
# get difference
block_means = all_routes_gdf.groupby(['from_block_geoid', 'method']).mean(numeric_only=True).reset_index()

# pivot to get length and risk methods as separate columns
pivoted = block_means.pivot(
    index='from_block_geoid', 
    columns='method',
    values='weighted_mean_score'
).reset_index()

# calculate the difference (length - risk)
pivoted['mean_score_diff'] = pivoted['length'] - pivoted['risk']

# merge back with the original data
block_means = block_means.merge(
    pivoted[['from_block_geoid', 'mean_score_diff']], 
    on='from_block_geoid'
).sort_values('mean_score_diff', ascending=False)

block_means.head()

,from_block_geoid,method,weighted_mean_score,min_score,max_score,sum_length,mean_score_diff
296,250173502022002,length,2.686718,1.227778,3.611111,1776.480302,1.483717
297,250173502022002,risk,1.203001,0.288889,3.250000,2266.276547,1.483717
315,250173502022011,risk,1.168207,0.155556,3.352778,2191.834037,1.426661
314,250173502022011,length,2.594868,1.150000,3.569444,1676.307006,1.426661
312,250173502022010,length,2.650912,1.150000,3.569444,1694.586087,1.387298


In [40]:
# combine top 3 and bottom 3
top_three = pd.DataFrame(block_means.head(6))
top_three['class'] = 'top_three'

bottom_three = pd.DataFrame(block_means.tail(6))
bottom_three['class'] = 'bottom_three'

block_means_short = pd.concat([top_three, bottom_three])

In [41]:
# save to csv
block_means_short.to_csv(OUT_PATH / "block_means_short.csv")

In [42]:
# Create the bar chart with sorting by mean_score_diff
chart = alt.Chart(block_means_short).mark_bar().encode(
    x=alt.X('from_block_geoid:N', 
            title='Block', 
            axis=alt.Axis(labelAngle=-45),
            sort=alt.EncodingSortField(field='mean_score_diff', op='max', order='descending')),
    y=alt.Y('weighted_mean_score:Q', title='Weighted Mean Score'),
    color=alt.Color('method:N', title='Method', scale=alt.Scale(scheme='set2')),
    xOffset='method:N',  # This creates grouped bars
    tooltip=[
        alt.Tooltip('from_block_geoid:N', title='Block'),
        alt.Tooltip('method:N', title='Method'),
        alt.Tooltip('weighted_mean_score:Q', title='Weighted Mean Score', format='.3f'),
        alt.Tooltip('sum_length:Q', title='Total Length', format='.1f')
    ]
).properties(
    width=600,
    height=400,
    title='Mean Composite Score by Block and Method (Risk vs. Length)'
)

# Get one row per block for the diff labels (take the first method's row since diff is the same)
block_diff = block_means_short.groupby('from_block_geoid').agg({
    'mean_score_diff': 'first',
    'weighted_mean_score': 'max'  # Use max to position text above tallest bar
}).reset_index()

# Add text labels centered above both bars showing mean_score_diff
text = alt.Chart(block_diff).mark_text(
    align='center',
    baseline='bottom',
    dy=-5,
    fontSize=10,
    fontWeight='bold'
).encode(
    x=alt.X('from_block_geoid:N', sort=alt.EncodingSortField(field='mean_score_diff', op='max', order='descending')),
    y=alt.Y('weighted_mean_score:Q'),
    text=alt.Text('mean_score_diff:Q', format='.3f')
)

# Add vertical line between 3rd and 4th blocks using pixel coordinates
vertical_line = alt.Chart(pd.DataFrame({'x': [1]})).mark_rule(
    color='gray',
    strokeDash=[5, 5],
    strokeWidth=2
).encode(
    x=alt.value(300),  # Adjust this pixel value to position the line (chart width is 600)
    y=alt.value(0),
    y2=alt.value(400)
)

# Combine the chart, text, and vertical line
final_chart = chart + text + vertical_line
final_chart

alt.LayerChart(...)

In [43]:
# save HTML to upload dir
final_chart.save(OUT_PATH / "chart2.html")

### Recombine blocks with spatial

To show them on a map.

In [44]:
# reset - combine top 3 and bottom 3
top_three = pd.DataFrame(block_means.head(6))
top_three['class'] = 'top_three'
bottom_three = pd.DataFrame(block_means.tail(6))
bottom_three['class'] = 'bottom_three'
block_means_short = pd.concat([top_three, bottom_three])

In [45]:
# merge on geoid
block_means_short = block_means_short.merge(
    somerville_census_blocks[['GEOID20', 'geometry']], 
    left_on='from_block_geoid', 
    right_on='GEOID20', 
    how='left'
)

# we only need one row for each block (eg, risk, not risk & length) 
block_means_short[block_means_short['method'] == 'risk']

,from_block_geoid,method,weighted_mean_score,min_score,max_score,sum_length,mean_score_diff,class,GEOID20,geometry
1,250173502022002,risk,1.203001,0.288889,3.250000,2266.276547,1.483717,top_three,250173502022002,POINT (-71.10073 42.3916)
2,250173502022011,risk,1.168207,0.155556,3.352778,2191.834037,1.426661,top_three,250173502022011,POINT (-71.09852 42.3898)
5,250173502022010,risk,1.263614,0.155556,3.416667,2266.515933,1.387298,top_three,250173502022010,POINT (-71.09907 42.39017)
7,250173501091004,risk,2.302554,0.644444,3.752778,3055.841249,0.318368,bottom_three,250173501091004,POINT (-71.08831 42.39519)
8,250173501082007,risk,2.130589,0.850000,3.477778,2447.608444,0.279777,bottom_three,250173501082007,POINT (-71.09523 42.39544)
10,250173501091010,risk,2.356694,0.727778,3.655556,2850.524714,0.232523,bottom_three,250173501091010,POINT (-71.09009 42.39443)


In [46]:
block_means_short = gpd.GeoDataFrame(block_means_short)

In [47]:
block_means_short.to_file(OUT_PATH / "block_means_short.gpkg", driver="GPKG")